In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1, "computer": 2}
# 2 words in vocab, 5 dimensional embeddings
embeds = nn.Embedding(3, 5)

In [3]:
for params in embeds.parameters():
    print(params)
    print(params.shape)

lookup_tensor = torch.tensor([word_to_ix["computer"]], dtype=torch.long)
print()
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631],
        [-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]], requires_grad=True)
torch.Size([3, 5])

tensor([2])
tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]],
       grad_fn=<EmbeddingBackward>)


In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])\
            for i in range(len(test_sentence) - 2)]
                

In [5]:
print(trigrams[:10])
print(len(trigrams))

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,'), (['thy', 'brow,'], 'And'), (['brow,', 'And'], 'dig'), (['And', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in')]
113


In [7]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(word_to_ix)
print('voab size: ', len(vocab))

{'to': 0, 'own': 1, 'art': 2, "beauty's": 3, 'How': 4, 'small': 5, 'Were': 6, 'where': 7, 'all': 8, 'If': 9, 'child': 10, "'This": 11, 'days;': 12, 'gazed': 13, 'new': 14, 'by': 15, 'old': 16, 'and': 17, 'of': 18, 'when': 19, 'thine': 20, 'shame,': 21, 'within': 22, 'To': 23, 'couldst': 24, 'lusty': 25, 'Shall': 26, 'worth': 27, 'Where': 28, 'trenches': 29, 'be': 30, 'shall': 31, 'all-eating': 32, 'winters': 33, 'thriftless': 34, 'field,': 35, 'livery': 36, 'now,': 37, 'succession': 38, 'thou': 39, 'see': 40, 'count,': 41, 'so': 42, 'eyes,': 43, 'Then': 44, 'being': 45, 'praise.': 46, 'thine!': 47, 'Proving': 48, 'beauty': 49, 'my': 50, 'thy': 51, 'were': 52, 'brow,': 53, 'cold.': 54, 'When': 55, 'weed': 56, 'on': 57, 'the': 58, 'more': 59, 'answer': 60, 'held:': 61, 'sum': 62, 'This': 63, 'treasure': 64, 'lies,': 65, 'old,': 66, "deserv'd": 67, "totter'd": 68, 'make': 69, 'an': 70, 'his': 71, 'sunken': 72, 'it': 73, 'deep': 74, 'mine': 75, 'use,': 76, 'blood': 77, 'in': 78, 'Thy': 79,

In [14]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
#         print(f'inputs: {inputs}')
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
#         print(f'out shape: {out.shape}')
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [16]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [20]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        # Step 1. Prepare the inputs to be passed to the model
        # (i.e turn the words into integer indices and wrap them
        # in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context],
                                   dtype = torch.long)
        
        # Step 2. Recall that torch *accumulates* gradients. Before
        # passing in a new instance you need to zero out the gradients
        # from the old instance
        model.zero_grad()
        
        # Step 3. Run the forward pass, getting log probabilities over
        # next words
        log_probs = model(context_idxs)
        
        # Step 4. Compute the loss function. (Again, Torch wants the)
        # target word rapped in a tensor
#         tens = torch.tensor([word_to_ix[target]], dtype=torch.long)
#         print(f'tens:{tens}, shape: {tens.shape}')
#         print('Log probs',log_probs.shape)
        
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]],
                                                    dtype=torch.long))
        
        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
#         print(f'epoch: {epoch}, loss: {loss.item():.4f}')
        
#         if epoch % 200 == 0:
#             print(f'epoch: {epoch}, loss: {loss.item():.4f}')
        
        # Get the python number from a 1-element Tensor by calling
        # tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

In [23]:
# The loss decreased every iteration over the training data!
print(losses)

[524.7347674369812, 500.12683844566345, 476.78927278518677, 453.9670879840851, 431.11694955825806, 407.77357625961304, 383.54322135448456, 358.36046224832535, 332.20822328329086, 305.332987844944]


In [26]:
ix_to_word = {ix:word for ix, word in enumerate(vocab)}
# print(ix_to_word)
context = ['when', 'forty']
context_vector = torch.tensor([word_to_ix[w] for w in context], dtype = torch.long)
a = model(context_vector)
# print(a)
#Print result
print(f'Raw text: {" ".join(test_sentence)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: When forty winters shall besiege thy brow, And dig deep trenches in thy beauty's field, Thy youth's proud livery so gazed on now, Will be a totter'd weed of small worth held: Then being asked, where all thy beauty lies, Where all the treasure of thy lusty days; To say, within thine own deep sunken eyes, Were an all-eating shame, and thriftless praise. How much more praise deserv'd thy beauty's use, If thou couldst answer 'This fair child of mine Shall sum my count, and make my old excuse,' Proving his beauty by succession thine! This were to be new made when thou art old, And see thy blood warm when thou feel'st it cold.

Context: ['when', 'forty']

Prediction: of
